In [9]:
import enum
import numpy as np
from typing import List, Dict, Optional, Tuple, Callable, Union

class ActionType(enum.IntEnum):
    PASS = 0
    # BET actions will be numbers from 1 to 100

class PlayerType(enum.IntEnum):
    CHANCE = -2  # For card dealing
    TERMINAL = -1  # Game is over
    # Regular players are 0, 1, 2, ...

class KuhnPoker:
    """Implementation of Kuhn Poker with variable betting amounts."""

    # Game-specific constants
    ANTE = 1
    INVALID_PLAYER = -1
    MIN_BET = 1
    MAX_BET = 100

    def __init__(self, num_players: int = 2):
        """Initialize a new game of Kuhn Poker.
        
        Args:
            num_players: Number of players (default: 2)
        """
        if not (2 <= num_players <= 10):
            raise ValueError("Number of players must be between 2 and 10")
            
        self.num_players = num_players
        self.reset()

    def reset(self):
        """Reset the game state to the beginning of a new game."""
        self.history = []  # List of (player, action) tuples
        self.first_bettor = self.INVALID_PLAYER
        self.card_dealt = [self.INVALID_PLAYER] * (self.num_players + 1)
        self.winner = self.INVALID_PLAYER
        self.pot = self.ANTE * self.num_players
        self.ante = [self.ANTE] * self.num_players
        self.current_bet = 0  # Track the current bet that needs to be called
        self.player_bets = [0] * self.num_players  # Track each player's total betting amount

    def current_player(self) -> int:
        """Returns the current player's id, or special values for chance/terminal."""
        if self.is_terminal():
            return PlayerType.TERMINAL
        
        if len(self.history) < self.num_players:
            return PlayerType.CHANCE
        else:
            return len(self.history) % self.num_players

    def get_player_card(self, player: int) -> Optional[int]:
        """Get the card dealt to a specific player."""
        if len(self.history) <= player:
            return None
        return self.history[player][1]  # Action in chance nodes represents the card

    def is_chance_node(self) -> bool:
        """Returns True if cards are still being dealt."""
        return len(self.history) < self.num_players

    def is_terminal(self) -> bool:
        """Returns True if the game is over."""
        return self.winner != self.INVALID_PLAYER

    def legal_actions(self) -> List[int]:
        """Returns a list of legal actions for the current player.
        
        Returns:
            - If dealing cards: list of undealt card values
            - If player acting: [PASS] + list of valid bet amounts
            - If game over: empty list
        """
        if self.is_terminal():
            return []
        
        if self.is_chance_node():
            return [card for card in range(len(self.card_dealt)) 
                   if self.card_dealt[card] == self.INVALID_PLAYER]
        else:
            # Always allow PASS (fold/check)
            actions = [ActionType.PASS]
            
            # Add valid bet amounts
            current_player = self.current_player()
            min_raise = max(self.MIN_BET, self.current_bet + 1)
            
            # If there's a current bet, player must at least call that amount
            if self.current_bet > 0:
                min_raise = self.current_bet
                
            # Add all valid bet amounts
            actions.extend(range(min_raise, self.MAX_BET + 1))
            
            return actions

    def apply_action(self, action: int):
        """Apply an action to the game state.
        
        Args:
            action: 
                - If at chance node: the card value being dealt
                - If at player node: 
                    0 for PASS (fold/check)
                    1-100 for betting that amount
        """
        current_player = self.current_player()
        
        if current_player == PlayerType.CHANCE:
            # Dealing a card
            player_receiving_card = len(self.history)
            self.card_dealt[action] = player_receiving_card
        else:
            # Player betting action
            if action > ActionType.PASS:  # Betting
                bet_amount = action
                if self.first_bettor == self.INVALID_PLAYER:
                    self.first_bettor = current_player
                
                # Update pot and player bets
                self.pot += bet_amount
                self.player_bets[current_player] += bet_amount
                self.current_bet = bet_amount
                self.ante[current_player] += bet_amount

        self.history.append((current_player, action))
        
        # Check for game end
        num_actions = len(self.history) - self.num_players
        
        if self.should_end_game():
            self.determine_winner()

    def should_end_game(self) -> bool:
        """Determines if the game should end based on the current state."""
        if len(self.history) <= self.num_players:
            return False
            
        num_actions = len(self.history) - self.num_players
        
        # Game ends if everyone passed
        if self.first_bettor == self.INVALID_PLAYER and num_actions == self.num_players:
            return True
            
        # Game ends if everyone responded to the first bet
        if (self.first_bettor != self.INVALID_PLAYER and 
            num_actions == self.num_players + self.first_bettor):
            return True
            
        # Game ends if someone passed after a bet
        if self.first_bettor != self.INVALID_PLAYER:
            last_action = self.history[-1][1]
            if last_action == ActionType.PASS:
                return True
                
        return False

    def determine_winner(self):
        """Determines the winner of the game."""
        if self.first_bettor == self.INVALID_PLAYER:
            # Nobody bet - highest card wins
            self.winner = self.card_dealt[self.num_players]
            if self.winner == self.INVALID_PLAYER:
                self.winner = self.card_dealt[self.num_players - 1]
        else:
            # Someone bet - highest remaining card among players who didn't fold wins
            for card in range(self.num_players, -1, -1):
                player = self.card_dealt[card]
                if player != self.INVALID_PLAYER and self.player_bets[player] == self.current_bet:
                    self.winner = player
                    break

    def returns(self) -> List[float]:
        """Returns a list of payoffs for each player."""
        if not self.is_terminal():
            return [0.0] * self.num_players
        
        returns = []
        for player in range(self.num_players):
            bet = self.player_bets[player] + self.ANTE
            returns.append(self.pot - bet if player == self.winner else -bet)
        return returns

    def __str__(self) -> str:
        """Returns a string representation of the current state."""
        # Show dealt cards
        dealt = " ".join(
            f"P{i}:{h[1]}" for i, h in enumerate(self.history[:min(len(self.history), self.num_players)])
        )
        
        # Show betting amounts
        if len(self.history) > self.num_players:
            betting = " Bets:" + " ".join(
                f"P{h[0]}:{'fold' if h[1] == 0 else h[1]}"
                for h in self.history[self.num_players:]
            )
            return dealt + betting
            
        return dealt


In [13]:
def example_game():
    """Shows a complete example game sequence with variable betting."""
    game = KuhnPoker(2)
    
    # Dealing phase
    print("Initial state:", game)
    print("Current player (CHANCE):", game.current_player())
    
    # Deal cards
    game.apply_action(1)  # Give card 1 to player 0
    game.apply_action(2)  # Give card 2 to player 1
    print("After dealing:", game)
    
    # Betting phase
    print("\nPlayer 0's turn:", game.current_player())
    print("Legal actions:", game.legal_actions())
    game.apply_action(20)  # Player 0 bets 20
    print("After Player 0 bets 20:", game)
    
    print("\nPlayer 1's turn:", game.current_player())
    print("Legal actions:", game.legal_actions())
    game.apply_action(50)  # Player 1 raises to 50
    print("After Player 1 raises to 50:", game)
    
    print("\nPlayer 0's turn:", game.current_player())
    print("Legal actions:", game.legal_actions())
    game.apply_action(ActionType.PASS)  # Player 0 folds
    print("After Player 0 folds:", game)
    
    print("\nGame over, returns:", game.returns())

    print("Winner is",game.winner)
    return game

In [14]:
example_game()


Initial state: 
Current player (CHANCE): -2
After dealing: P0:1 P1:2

Player 0's turn: 0
Legal actions: [<ActionType.PASS: 0>, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
After Player 0 bets 20: P0:1 P1:2 Bets:P0:20

Player 1's turn: 1
Legal actions: [<ActionType.PASS: 0>, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
After Player 1 raises to 50: P0:1 P1:2 Bets:P0:20 P1: